In [55]:
#play around with insightface model.
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from lfw import gen_all_pic_labels
from sklearn.cluster import DBSCAN
from insightface.data import get_image as ins_get_image
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS
import umap
from mpl_toolkits.mplot3d import Axes3D


## Load in of the pics-files pairs

In [2]:
pics,names = gen_all_pic_labels('./lfw_funneled/')
df = pd.DataFrame({'files':pics,'name':names})

In [3]:
#namely we have 5749 different faces
df.name.value_counts()

George_W_Bush        530
Colin_Powell         236
Tony_Blair           144
Donald_Rumsfeld      121
Gerhard_Schroeder    109
                    ... 
Paul_Walker            1
Ewan_McGregor          1
James_Brazelton        1
Ed_Book                1
Michel_Minard          1
Name: name, Length: 5749, dtype: int64

In [4]:
names = pd.DataFrame(df.name.value_counts())
names = pd.DataFrame(names[names.name>1])
names = list(names.index)

## Thus, we have exactly 1680 duplicated faces

# We filter these duplicated faces for cluster test propose

In [12]:
df_dup = df[df['name'].isin(names)]
df_dup.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9165 entries, 8 to 13231
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   files   9165 non-null   object
 1   name    9165 non-null   object
dtypes: object(2)
memory usage: 214.8+ KB


In [13]:
df_dup.name.value_counts()

George_W_Bush        530
Colin_Powell         236
Tony_Blair           144
Donald_Rumsfeld      121
Gerhard_Schroeder    109
                    ... 
Stanley_Tong           2
Lindsay_Benko          2
Raquel_Welch           2
Anthony_Fauci          2
John_Garamendi         2
Name: name, Length: 1680, dtype: int64

In [14]:
#number of points we are going to cluster
num = 1000

In [15]:
#shuffle the dataset
df_dup = df_dup.sample(frac=1).reset_index()

In [16]:
df_dup = df_dup[0:num]

In [22]:
df_dup

,index,files,name
0,13053,./lfw_funneled/John_Ashcroft/John_Ashcroft_004...,John_Ashcroft
1,6916,./lfw_funneled/Alan_Greenspan/Alan_Greenspan_0...,Alan_Greenspan
2,7655,./lfw_funneled/Colin_Powell/Colin_Powell_0087.jpg,Colin_Powell
3,357,./lfw_funneled/Jennifer_Aniston/Jennifer_Anist...,Jennifer_Aniston
4,2516,./lfw_funneled/Carson_Palmer/Carson_Palmer_000...,Carson_Palmer
...,...,...,...
995,5638,./lfw_funneled/Pierce_Brosnan/Pierce_Brosnan_0...,Pierce_Brosnan
996,5079,./lfw_funneled/Ruth_Dreifuss/Ruth_Dreifuss_000...,Ruth_Dreifuss
997,8464,./lfw_funneled/Alastair_Campbell/Alastair_Camp...,Alastair_Campbell
998,7580,./lfw_funneled/Colin_Powell/Colin_Powell_0024.jpg,Colin_Powell


In [23]:
df_dup = df_dup.reset_index()

In [25]:
print(df_dup.name.value_counts())

George_W_Bush         63
Colin_Powell          30
Tony_Blair            19
Gerhard_Schroeder     14
Donald_Rumsfeld       13
                      ..
Nick_Nolte             1
Toni_Braxton           1
Jean-David_Levitte     1
Al_Gore                1
Ruth_Dreifuss          1
Name: name, Length: 584, dtype: int64


In [26]:
#prepare for the face recognizing
embs = np.zeros((num,512))
app = FaceAnalysis(name='buffalo_l')
label = []
app.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/mafffia/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/mafffia/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/mafffia/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/mafffia/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/mafffia/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

In [27]:
# pca = PCA(n_components=50)
# newX = pca.fit_transform(X) 
for i in range(num):
    pic = df_dup.loc[i]['files']
    name = df_dup.loc[i]['name']
    label.append(name)
    print(pic,name)
    img = cv2.imread(pic)
    try:
        face_emb = app.get(img)[0]['embedding']
        embs[i,:] = face_emb[:]
    except:
        embs[i,:] = np.zeros((512))

./lfw_funneled/John_Ashcroft/John_Ashcroft_0044.jpg John_Ashcroft
./lfw_funneled/Alan_Greenspan/Alan_Greenspan_0001.jpg Alan_Greenspan
./lfw_funneled/Colin_Powell/Colin_Powell_0087.jpg Colin_Powell
./lfw_funneled/Jennifer_Aniston/Jennifer_Aniston_0014.jpg Jennifer_Aniston
./lfw_funneled/Carson_Palmer/Carson_Palmer_0001.jpg Carson_Palmer
./lfw_funneled/Jean_Chretien/Jean_Chretien_0003.jpg Jean_Chretien
./lfw_funneled/Kathleen_Glynn/Kathleen_Glynn_0001.jpg Kathleen_Glynn
./lfw_funneled/Mel_Brooks/Mel_Brooks_0001.jpg Mel_Brooks
./lfw_funneled/Jose_Serra/Jose_Serra_0008.jpg Jose_Serra
./lfw_funneled/Kim_Clijsters/Kim_Clijsters_0014.jpg Kim_Clijsters
./lfw_funneled/Jane_Pauley/Jane_Pauley_0002.jpg Jane_Pauley
./lfw_funneled/Holly_Hunter/Holly_Hunter_0002.jpg Holly_Hunter
./lfw_funneled/Brooke_Shields/Brooke_Shields_0001.jpg Brooke_Shields
./lfw_funneled/Ariel_Sharon/Ariel_Sharon_0005.jpg Ariel_Sharon
./lfw_funneled/Pervez_Musharraf/Pervez_Musharraf_0018.jpg Pervez_Musharraf
./lfw_funneled/L

./lfw_funneled/Ariel_Sharon/Ariel_Sharon_0065.jpg Ariel_Sharon
./lfw_funneled/Clare_Short/Clare_Short_0004.jpg Clare_Short
./lfw_funneled/George_W_Bush/George_W_Bush_0193.jpg George_W_Bush
./lfw_funneled/Roh_Moo-hyun/Roh_Moo-hyun_0008.jpg Roh_Moo-hyun
./lfw_funneled/Mike_Scioscia/Mike_Scioscia_0001.jpg Mike_Scioscia
./lfw_funneled/Dick_Cheney/Dick_Cheney_0013.jpg Dick_Cheney
./lfw_funneled/Toshihiko_Fukui/Toshihiko_Fukui_0003.jpg Toshihiko_Fukui
./lfw_funneled/Oleksandr_Moroz/Oleksandr_Moroz_0002.jpg Oleksandr_Moroz
./lfw_funneled/Laura_Hernandez/Laura_Hernandez_0001.jpg Laura_Hernandez
./lfw_funneled/Sue_Wicks/Sue_Wicks_0001.jpg Sue_Wicks
./lfw_funneled/Gonzalo_Sanchez_de_Lozada/Gonzalo_Sanchez_de_Lozada_0002.jpg Gonzalo_Sanchez_de_Lozada
./lfw_funneled/Bruce_Springsteen/Bruce_Springsteen_0002.jpg Bruce_Springsteen
./lfw_funneled/Ann_Veneman/Ann_Veneman_0005.jpg Ann_Veneman
./lfw_funneled/Paul_Bremer/Paul_Bremer_0016.jpg Paul_Bremer
./lfw_funneled/George_W_Bush/George_W_Bush_0352.jpg 

./lfw_funneled/David_Nalbandian/David_Nalbandian_0014.jpg David_Nalbandian
./lfw_funneled/Gerhard_Schroeder/Gerhard_Schroeder_0075.jpg Gerhard_Schroeder
./lfw_funneled/George_W_Bush/George_W_Bush_0527.jpg George_W_Bush
./lfw_funneled/George_W_Bush/George_W_Bush_0333.jpg George_W_Bush
./lfw_funneled/Johnny_Depp/Johnny_Depp_0001.jpg Johnny_Depp
./lfw_funneled/Junichiro_Koizumi/Junichiro_Koizumi_0059.jpg Junichiro_Koizumi
./lfw_funneled/Atal_Bihari_Vajpayee/Atal_Bihari_Vajpayee_0013.jpg Atal_Bihari_Vajpayee
./lfw_funneled/Gerard_Depardieu/Gerard_Depardieu_0001.jpg Gerard_Depardieu
./lfw_funneled/Donald_Rumsfeld/Donald_Rumsfeld_0015.jpg Donald_Rumsfeld
./lfw_funneled/Colin_Powell/Colin_Powell_0077.jpg Colin_Powell
./lfw_funneled/Dominique_de_Villepin/Dominique_de_Villepin_0014.jpg Dominique_de_Villepin
./lfw_funneled/Alvaro_Uribe/Alvaro_Uribe_0006.jpg Alvaro_Uribe
./lfw_funneled/Rogerio_Romero/Rogerio_Romero_0001.jpg Rogerio_Romero
./lfw_funneled/Gerhard_Schroeder/Gerhard_Schroeder_0004.jp

./lfw_funneled/David_Trimble/David_Trimble_0001.jpg David_Trimble
./lfw_funneled/Jean_Charest/Jean_Charest_0017.jpg Jean_Charest
./lfw_funneled/Steven_Spielberg/Steven_Spielberg_0004.jpg Steven_Spielberg
./lfw_funneled/Lord_Hutton/Lord_Hutton_0002.jpg Lord_Hutton
./lfw_funneled/Carlos_Quintanilla_Schmidt/Carlos_Quintanilla_Schmidt_0001.jpg Carlos_Quintanilla_Schmidt
./lfw_funneled/Vicente_Fox/Vicente_Fox_0026.jpg Vicente_Fox
./lfw_funneled/Nabil_Shaath/Nabil_Shaath_0003.jpg Nabil_Shaath
./lfw_funneled/Colin_Powell/Colin_Powell_0175.jpg Colin_Powell
./lfw_funneled/Justine_Henin/Justine_Henin_0001.jpg Justine_Henin
./lfw_funneled/Ray_Romano/Ray_Romano_0008.jpg Ray_Romano
./lfw_funneled/George_W_Bush/George_W_Bush_0187.jpg George_W_Bush
./lfw_funneled/Junichiro_Koizumi/Junichiro_Koizumi_0055.jpg Junichiro_Koizumi
./lfw_funneled/Laura_Bush/Laura_Bush_0023.jpg Laura_Bush
./lfw_funneled/Jose_Maria_Aznar/Jose_Maria_Aznar_0022.jpg Jose_Maria_Aznar
./lfw_funneled/Mike_Weir/Mike_Weir_0010.jpg Mi

./lfw_funneled/Carlos_Menem/Carlos_Menem_0013.jpg Carlos_Menem
./lfw_funneled/Gunter_Pleuger/Gunter_Pleuger_0007.jpg Gunter_Pleuger
./lfw_funneled/Lleyton_Hewitt/Lleyton_Hewitt_0024.jpg Lleyton_Hewitt
./lfw_funneled/Johnny_Carson/Johnny_Carson_0001.jpg Johnny_Carson
./lfw_funneled/Bill_Frist/Bill_Frist_0004.jpg Bill_Frist
./lfw_funneled/Ariel_Sharon/Ariel_Sharon_0050.jpg Ariel_Sharon
./lfw_funneled/Colin_Powell/Colin_Powell_0136.jpg Colin_Powell
./lfw_funneled/Mark_Hamister/Mark_Hamister_0001.jpg Mark_Hamister
./lfw_funneled/Edward_Lu/Edward_Lu_0003.jpg Edward_Lu
./lfw_funneled/Guillermo_Canas/Guillermo_Canas_0001.jpg Guillermo_Canas
./lfw_funneled/Jean-Pierre_Raffarin/Jean-Pierre_Raffarin_0007.jpg Jean-Pierre_Raffarin
./lfw_funneled/Tiger_Woods/Tiger_Woods_0011.jpg Tiger_Woods
./lfw_funneled/Spencer_Abraham/Spencer_Abraham_0007.jpg Spencer_Abraham
./lfw_funneled/Sally_Kirkland/Sally_Kirkland_0003.jpg Sally_Kirkland
./lfw_funneled/Martina_McBride/Martina_McBride_0003.jpg Martina_McBrid

./lfw_funneled/Bill_Gates/Bill_Gates_0013.jpg Bill_Gates
./lfw_funneled/Jesse_Jackson/Jesse_Jackson_0008.jpg Jesse_Jackson
./lfw_funneled/Al_Sharpton/Al_Sharpton_0004.jpg Al_Sharpton
./lfw_funneled/Amelie_Mauresmo/Amelie_Mauresmo_0004.jpg Amelie_Mauresmo
./lfw_funneled/Sergei_Ivanov/Sergei_Ivanov_0003.jpg Sergei_Ivanov
./lfw_funneled/Tony_Blair/Tony_Blair_0048.jpg Tony_Blair
./lfw_funneled/Serena_Williams/Serena_Williams_0014.jpg Serena_Williams
./lfw_funneled/Juan_Carlos_Ferrero/Juan_Carlos_Ferrero_0018.jpg Juan_Carlos_Ferrero
./lfw_funneled/Carlos_Vives/Carlos_Vives_0004.jpg Carlos_Vives
./lfw_funneled/Nestor_Kirchner/Nestor_Kirchner_0021.jpg Nestor_Kirchner
./lfw_funneled/David_Beckham/David_Beckham_0016.jpg David_Beckham
./lfw_funneled/Boris_Becker/Boris_Becker_0005.jpg Boris_Becker
./lfw_funneled/Mario_Dumont/Mario_Dumont_0001.jpg Mario_Dumont
./lfw_funneled/Junichiro_Koizumi/Junichiro_Koizumi_0024.jpg Junichiro_Koizumi
./lfw_funneled/Magui_Serna/Magui_Serna_0001.jpg Magui_Serna
.

./lfw_funneled/Donald_Rumsfeld/Donald_Rumsfeld_0108.jpg Donald_Rumsfeld
./lfw_funneled/Lee_Jun/Lee_Jun_0001.jpg Lee_Jun
./lfw_funneled/Jean_Chretien/Jean_Chretien_0040.jpg Jean_Chretien
./lfw_funneled/Maria_Shriver/Maria_Shriver_0003.jpg Maria_Shriver
./lfw_funneled/Stephen_Friedman/Stephen_Friedman_0001.jpg Stephen_Friedman
./lfw_funneled/Julio_Iglesias_Jr/Julio_Iglesias_Jr_0002.jpg Julio_Iglesias_Jr
./lfw_funneled/Jim_Hahn/Jim_Hahn_0004.jpg Jim_Hahn
./lfw_funneled/Amelia_Vega/Amelia_Vega_0003.jpg Amelia_Vega
./lfw_funneled/Aaron_Peirsol/Aaron_Peirsol_0004.jpg Aaron_Peirsol
./lfw_funneled/Eric_Clapton/Eric_Clapton_0002.jpg Eric_Clapton
./lfw_funneled/Hu_Jintao/Hu_Jintao_0001.jpg Hu_Jintao
./lfw_funneled/Condoleezza_Rice/Condoleezza_Rice_0004.jpg Condoleezza_Rice
./lfw_funneled/Colin_Powell/Colin_Powell_0170.jpg Colin_Powell
./lfw_funneled/John_Ruiz/John_Ruiz_0002.jpg John_Ruiz
./lfw_funneled/Adam_Sandler/Adam_Sandler_0004.jpg Adam_Sandler
./lfw_funneled/Ariel_Sharon/Ariel_Sharon_0015.

./lfw_funneled/Norah_Jones/Norah_Jones_0001.jpg Norah_Jones
./lfw_funneled/Mike_Weir/Mike_Weir_0006.jpg Mike_Weir
./lfw_funneled/Roh_Moo-hyun/Roh_Moo-hyun_0001.jpg Roh_Moo-hyun
./lfw_funneled/Tony_Blair/Tony_Blair_0130.jpg Tony_Blair
./lfw_funneled/Mike_Montgomery/Mike_Montgomery_0002.jpg Mike_Montgomery
./lfw_funneled/Hilmi_Ozkok/Hilmi_Ozkok_0002.jpg Hilmi_Ozkok
./lfw_funneled/Angelina_Jolie/Angelina_Jolie_0013.jpg Angelina_Jolie
./lfw_funneled/Muhammad_Ali/Muhammad_Ali_0007.jpg Muhammad_Ali
./lfw_funneled/Lleyton_Hewitt/Lleyton_Hewitt_0021.jpg Lleyton_Hewitt
./lfw_funneled/Gerhard_Schroeder/Gerhard_Schroeder_0059.jpg Gerhard_Schroeder
./lfw_funneled/Vitali_Klitschko/Vitali_Klitschko_0002.jpg Vitali_Klitschko
./lfw_funneled/Jose_Serra/Jose_Serra_0002.jpg Jose_Serra
./lfw_funneled/Choi_Sung-hong/Choi_Sung-hong_0004.jpg Choi_Sung-hong
./lfw_funneled/David_Wells/David_Wells_0002.jpg David_Wells
./lfw_funneled/Gerhard_Schroeder/Gerhard_Schroeder_0046.jpg Gerhard_Schroeder
./lfw_funneled/I

./lfw_funneled/Martin_Scorsese/Martin_Scorsese_0001.jpg Martin_Scorsese
./lfw_funneled/Cathy_Freeman/Cathy_Freeman_0001.jpg Cathy_Freeman
./lfw_funneled/Sergio_Vieira_De_Mello/Sergio_Vieira_De_Mello_0009.jpg Sergio_Vieira_De_Mello
./lfw_funneled/Amelia_Vega/Amelia_Vega_0002.jpg Amelia_Vega
./lfw_funneled/Padraig_Harrington/Padraig_Harrington_0003.jpg Padraig_Harrington
./lfw_funneled/Svetlana_Koroleva/Svetlana_Koroleva_0002.jpg Svetlana_Koroleva
./lfw_funneled/Tom_Cruise/Tom_Cruise_0002.jpg Tom_Cruise
./lfw_funneled/Donald_Rumsfeld/Donald_Rumsfeld_0094.jpg Donald_Rumsfeld
./lfw_funneled/Michelle_Branch/Michelle_Branch_0001.jpg Michelle_Branch
./lfw_funneled/Nora_Bendijo/Nora_Bendijo_0001.jpg Nora_Bendijo
./lfw_funneled/Isabelle_Huppert/Isabelle_Huppert_0001.jpg Isabelle_Huppert
./lfw_funneled/Nan_Wang/Nan_Wang_0004.jpg Nan_Wang
./lfw_funneled/George_W_Bush/George_W_Bush_0233.jpg George_W_Bush
./lfw_funneled/Gary_Carter/Gary_Carter_0001.jpg Gary_Carter
./lfw_funneled/Winona_Ryder/Winona

## Clustering with DBSCAN

In [76]:
reducer = umap.UMAP(random_state=42,n_components=25)
face_umap = reducer.fit_transform(embs)
scan = DBSCAN(min_samples=2,eps=0.3).fit(face_umap)
clustering = list(scan.labels_)

out_name = []
out_clu = []

for i in range(len(clustering)):
    if(clustering[i]>=0):
        out_name.append(label[i])
        out_clu.append(clustering[i])

df_out = pd.DataFrame({'real':out_name,'cluster':out_clu})
pd.set_option('display.max_columns',1000)
df_out = df_out.sort_values(by='cluster')
# print(df_out.sort_values(by='real'))
for i in df_out.index:
    print(df_out.loc[i]['cluster'] , '  ' , df_out.loc[i]['real'])
print('total clustered points :%d'%len(out_name))

0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
1    Alan_Greenspan
1    Alan_Greenspan
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
2    Colin_Powell
3    Jean_Chretien
3    Jean_Chretien
3    Jean_Chretien
3    Jean_Chretien
3    Jean_Chretien
3    Jean_Chretien
3    Jean_Chretien
4    Jose_Serra
4    Jose_Serra
5    Kim_Clijsters
5    Kim_Clijsters
5    Kim_Clijsters
6    Ariel_Sharon
6    Ariel_Sharon
6    Ariel_Sharon
6    Ariel_Sharon
6    Ariel_Sharon
6    Ariel_Sharon
6    Ariel_S

158    Nora_Bendijo
158    Isabelle_Huppert
158    Nan_Wang
158    Gary_Carter
158    Kenneth_Bowersox
158    Bill_Simon
158    Brian_Heidik
158    Trent_Lott
158    Ruth_Dreifuss
158    Sila_Calderon
158    Kathleen_Kennedy_Townsend
158    Dennis_Kozlowski
158    Prince_Charles
158    Choi_Sung-hong
158    David_Wells
158    Ruben_Studdard
158    Lennox_Lewis
158    Derek_Lowe
158    Rita_Moreno
158    Vanessa_Williams
158    Martin_Hoellwarth
158    Bill_Simon
158    Mariangel_Ruiz_Torrealba
158    Neri_Marcore
158    Daniel_Radcliffe
158    Celine_Dion
158    Liza_Minnelli
158    Conchita_Martinez
158    Nancy_Reagan
158    Bill_Paxton
158    Hilmi_Ozkok
158    Vitali_Klitschko
158    Adolfo_Aguilar_Zinser
158    Claudia_Pechstein
158    Brian_Cowen
158    Ari_Fleischer
158    Jose_Sarney
158    Susan_Collins
158    Mark_Philippoussis
158    Penelope_Ann_Miller
158    Javier_Solana
158    Marissa_Jaret_Winokur
158    Azra_Akin
158    Pat_Burns
158    John_Jumper
158    Tim_Henman
15

## Clustering with OPTICS

In [71]:
reducer = umap.UMAP(random_state=42,n_components=30)
face_umap = reducer.fit_transform(embs)
scan = OPTICS(min_samples=2,metric='cosine').fit(face_umap)
clustering = list(scan.labels_)

out_name = []
out_clu = []

for i in range(len(clustering)):
    if(clustering[i]>=0):
        out_name.append(label[i])
        out_clu.append(clustering[i])

df_out = pd.DataFrame({'real':out_name,'cluster':out_clu})
pd.set_option('display.max_columns',1000)
df_out = df_out.sort_values(by='cluster')
# print(df_out.sort_values(by='real'))
for i in df_out.index:
    print(df_out.loc[i]['cluster'] , '  ' , df_out.loc[i]['real'])
print('total clustered points :%d'%len(out_name))

0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
0    John_Ashcroft
1    Amelie_Mauresmo
1    Amelie_Mauresmo
2    Pervez_Musharraf
2    Pervez_Musharraf
2    Pervez_Musharraf
3    Walter_Mondale
3    Walter_Mondale
4    Carlos_Moya
4    Carlos_Moya
5    Penelope_Ann_Miller
5    Emma_Watson
5    Svetlana_Koroleva
5    Michelle_Branch
6    Nora_Bendijo
6    Vin_Diesel
7    Ruth_Dreifuss
7    Nan_Wang
8    Zhang_Ziyi
8    Lucy_Liu
8    Jose_Sarney
9    Sila_Calderon
9    Martin_Scorsese
10    Gary_Carter
10    Trent_Lott
11    Bill_Simon
11    Brian_Heidik
11    Bill_Simon
12    Derek_Lowe
12    Raoul_Ruiz
12    Nancy_Reagan
13    Jiri_Novak
13    Saddam_Hussein
14    Ari_Fleischer
14    Marissa_Jaret_Winokur
15    Neri_Marcore
15    Muhammad_Saeed_al-Sahhaf
15    Norm_Coleman
16    David_Wells
16    Vanessa_Williams
16    Vitali_Klitschko
17    Dennis_Kozlowski
17    Daniel_Radcliffe
18    Matt_Doherty
18    Mark_Philippoussis
18    Pat_Burn

In [65]:
face_umap[:,1].reshape(1,-1).shape

(1, 1000)

In [73]:
scan.labels_.shape

(1000,)